# Adding tables to the view

In this notebook, we add tables to the `ipyaladin` view.

In [1]:
from ipyaladin import Aladin, CircleError, EllipseError
from astropy.table import QTable
import astropy.units as u
from astroquery.simbad import Simbad

## Table from a database

Let's query the [SIMBAD](https://simbad.cds.unistra.fr/simbad/ "https://simbad.cds.unistra.fr/simbad/") database around the Crab nebula.

In [2]:
table = Simbad.query_region("Messier 1", radius=0.03 * u.deg)
table

main_id,ra,dec,coo_err_maj,coo_err_min,coo_err_angle,coo_wavelength,coo_bibcode
,deg,deg,mas,mas,deg,,
object,float64,float64,float32,float32,int16,str1,object
FHES J0534.5+2201,83.6327596,22.0203278,--,--,--,G,2018ApJS..237...32A
PLCKERC -070 G184.55-05.78,83.625,22.016666666666666,--,--,--,F,2016MNRAS.458.3619C
SCOPE G184.56-05.78,83.635,22.009999999999998,--,--,--,S,2019MNRAS.485.2895E
SCOPE G184.56-05.79,83.629,22.012,--,--,--,S,2019MNRAS.485.2895E
JCMTSF J053427.4+220227,83.614,22.042000000000005,--,--,--,S,2019MNRAS.485.2895E
JCMTSF J053430.4+220151,83.628,22.034,--,--,--,S,2019MNRAS.485.2895E
SCOPE G184.54-05.77,83.634,22.039,--,--,--,S,2019MNRAS.485.2895E
SCOPE G184.55-05.76,83.65,22.032,--,--,--,S,2019MNRAS.485.2895E


Now we instantiate an `ipyaladin` view around the nebula.

In [3]:
aladin = Aladin(fov=0.4, target="Messier 1")
aladin

In `add_table`, the shape argument can take the values "square", "circle", "plus", "cross", "rhomb", or "triangle" for the simple shapes. 

The two specific classes of error shapes `EllipseError` and `CircleError` adapt the shapes drawn to the values of the errors around the coordinates.

SIMBAD gives errors as ellipses, so let's use an `EllipseError` shape:

In [4]:
aladin.add_table(
    table,
    shape=EllipseError(
        maj_axis="coo_err_maj",
        min_axis="coo_err_min",
        angle="coo_err_angle",
        default_shape="cross",
    ),
    color="lightskyblue",
    # source_size will only affect the sources which fallback to the default shape,
    # or the ones too small to be drawn
    source_size=20,
    name="SIMBAD query",
)

If you zoom in the view, you'll remark that the sources which had error information appear as ellipses. If if there is no information in the error columns, then the sources appear with the fallback shape `default_shape`. `default_shape` should be one of the strings allowed in `shape`.

If the ellipse would be too small to be drawn in the widget, then it appears as a square, see the source `2MASS J05343561+2200372` located in `83.64842925852 +22.01034559457`. If you zoom in close to this square, you'll see the square turning into an ellipse when it is big enough to be drawn.

## Displaying a QTable

Let's create a table from scratch and add it to the view. We will draw `CircleError` shapes this time.

In [5]:
t = QTable(
    {
        "error": [0.005, 0.002, 0.003] * u.deg,
        "coo1": [83.63451584700, 83.61368056017, 83.58780251600] * u.deg,
        "coo2": [22.05652591227, 21.97517807639, 21.99277764451] * u.deg,
        "name": ["Source 1", "Source 2", "Source 3"],
        "parallax": [1.7703, 0.5112, 0.3735] * u.mas,
    },
    meta={"name": "my_sample_table"},
)

aladin.add_table(
    t,
    name=t.meta["name"],
    shape=CircleError(
        radius="error",
        probability_threshold=0.989,  # roughly 3 sigma
    ),
    color="pink",
    ra_field="coo1",
    dec_field="coo2",
)

Note that we used unconventional column names for the coordinates fields here. In this case, we can specify the column names where Aladin Lite should read the coordinates via `ra_field` and `dec_field`.

We used a very high probability threshold. This corresponds to roughly 3 sigma in the drawn circle.